In [1]:
!pip install openpyxl
!pip install hazm

     |████████████████████████████████| 243 kB 420 kB/s 
     |████████████████████████████████| 316 kB 412 kB/s 
     |████████████████████████████████| 233 kB 3.7 MB/s 
     |████████████████████████████████| 1.4 MB 5.1 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394470 sha256=cfbe7d5ae6458ec8b8e4a831a5faa3c37e6494c7554de48f74e750c872d8bdf6
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=169454 sha256=b63d704b3344613c4122b93311fc6d5e9abb25188467ad56b44788b55e7b59e0
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into acco

In [2]:
import numpy as np 
import pandas as pd
from hazm import *
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/persian-digikala-reviwes/2-p9vcb5bb.xlsx


In [3]:
df = pd.read_excel('../input/persian-digikala-reviwes/2-p9vcb5bb.xlsx')
df.shape

(100000, 12)

In [4]:
normalizer = Normalizer()
df = df.dropna()

df['comment'] = df['comment'].apply(lambda x:normalizer.normalize(x))
df = df[['comment','likes','dislikes','recommend']]

df['comment'] = df['comment'].apply(lambda x: re.sub('[0-9]+', ' ', x))

df = df[((df['likes']/df['dislikes'])>1)&(df['recommend'] == "recommended")
       |((df['recommend'] == "not_recommended")&(df['dislikes']/df['likes']>1))]

To_Process = df[['recommend','comment']]
To_Process.shape

(7596, 2)

In [5]:
train, test = train_test_split(To_Process, test_size=0.2)

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Define the sequence lengths, max number of words and embedding dimensions
MAX_SEQUENCE_LENGTH = 300 #can plot a graph for length
MAX_NB_WORDS = 15000
EMBEDDING_DIM = 100

# Get the frequently occurring words
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train.comment)
train_sequences = tokenizer.texts_to_sequences(train.comment)
test_sequences = tokenizer.texts_to_sequences(test.comment)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

#some padding shit
train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 19086 unique tokens.


In [7]:
#some shit with labels

from sklearn.preprocessing import LabelEncoder

train_labels = train['recommend']
test_labels = test['recommend']

le = LabelEncoder()
le.fit(train_labels)
train_labels = le.transform(train_labels)
test_labels = le.transform(test_labels)
print(le.classes_)

['not_recommended' 'recommended']


In [8]:
from keras.utils import to_categorical

#change data types (actualy idk why)
labels_train = to_categorical(np.asarray(train_labels))
labels_test = to_categorical(np.asarray(test_labels))

In [9]:
from keras.layers import Sequential, Model, Dense, Input, LSTM, Embedding, Dropout, Activation, BatchNormalization, Flatten
model2 = Sequential()

#Embedding: batch_size*input_length --> batch_dimension(none)*input_length*output_dim(embbeding_dim)
model2.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model2.add(LSTM(units=16, activation='relu', recurrent_activation='hard_sigmoid',return_sequences=True))
model2.add(Dropout(0.2))
model2.add(BatchNormalization())
model2.add(Flatten())
model2.add(Dense(2,activation='softmax'))

model2.compile(loss = 'binary_crossentropy',optimizer='adam',metrics = ['accuracy'])
model2.fit(train_data, labels_train, batch_size=16, epochs=5, validation_data=(test_data, labels_test))

ImportError: cannot import name 'Sequential' from 'keras.layers' (/opt/conda/lib/python3.7/site-packages/keras/layers/__init__.py)

In [10]:
import sklearn
from sklearn.metrics import precision_recall_fscore_support as score

predicted=model2.predict(test_data)
predicted

precision, recall, fscore, support = score(labels_test, predicted.round())
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print("############################")
print(sklearn.metrics.classification_report(labels_test, predicted.round()))

NameError: name 'model2' is not defined

In [11]:
#example
k = 3
print(test.iloc[k])
x = model2(test_data[k:k+1,:])
x

recommend                                          recommended
comment      من واقعا ازش راضیم ولی تنها مشکلم گردنشه که هم...
Name: 48821, dtype: object


NameError: name 'model2' is not defined